### 라이브러리

In [47]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb

### 데이터 처리

In [48]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [49]:
# 요일별 랭크를 부여하여 설정하였다.
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [50]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['요일'] = test['일자'].dt.weekday

In [51]:
# 메뉴는 사용하지 않았다. 드랍

drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [52]:
# 본사정원수 변수 대신에 식사가능자수 변수를 만들어 사용하였다. 

train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

### 중식 월 랭크

```
train.query('년 == 2020').groupby('월').mean()['중식계'].sort_values()

중식 월
6      760.000000
1      811.000000
5      815.578947
8      825.100000
7      827.826087
10     852.684211
11     864.700000
9      871.650000
4      924.650000
2      937.842105
3      958.090909
12    1048.500000
Name: 중식계, dtype: float64

2021년의 1월은 약 1000. 1월을 2021년꺼를 사용한 이유는 2월부터 재택근무를 시작하여서 2020년 2월 이전과 이후의 양상이 다르다고 판단하였기에.
```

### 석식 월 랭크
```
train.groupby('월').mean()['석식계'].sort_values()

월
12    401.168317
1     424.564356
11    434.250000
7     437.300000
5     458.131313
8     459.485981
6     460.355556
9     463.617021
4     480.825243
10    481.641304
2     514.860215
3     530.700935
Name: 석식계, dtype: float64

석식의 경우는 전체에서 월별 평균순으로 랭크를 매겼음.
```

In [53]:
# 자기개발의 날 제거한 중식계의 월을 랭크매긴다. 
# (1월은 2021년의 1월, 2~12월은 2020년의 동일월을 기준으로함.)

month1 = {
    6:12,
	5:11,
	8:10,
	7:9	,
	10:8,
	11:7,
	9:6,
	4:5,
	2:4,
	3:3,
	1:2,
	12:1
}


# 석식계의 월을 랭크매긴다.
month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') #int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

### 공휴일전후 변수 사용
공휴일 전은 수가 적고, 공휴일 후는 많은 양상을 보였다.
그 중 수가 적은 공휴일만 사용함. (후의 경우도 수가 적은 양상을 보이는 값은 같이 추가하였음)
2의 경우는 황금연휴라 생각되는 부분을 적음

In [54]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [55]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1 # 더미코딩시 test에 공휴일전후_1이 생기도록 하기위해 잠시 설정후 0으로 바꿔줌 

In [56]:
# 원핫인코딩 사용
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])

test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0 # 다시 바꿔주었다.

In [57]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

478.8605851979346

In [58]:
# 코로나 이전에 자기개발의 날을 월말 수요일에 시행하였으나,  코로나 발생이후 2020년에는 2월과 11월 단 2회밖에
# 시행되지않음. 이에따라 테스트셋에서도 시행이 되지 않을것이라 판단하였다
# 하지만 모델에서는 있다고 판단하여 1/27이 100 단위가 나와서, 이를 막기위해 1월의 자기개발의 날에는
# 0이 아닌 값들의 석식계 평균을 대입해주었다

# # 현재 1월만 대체
# train['석식계'][244] = 478.8605851979346
# #train['석식계'][281] = 398
# train['석식계'][492] = 478.8605851979346
# #train['석식계'][502] = 398
# #train['석식계'][510] = 398
# #train['석식계'][529] = 398
# train['석식계'][730] = 478.8605851979346
# #train['석식계'][747] = 398
# #train['석식계'][766] = 398
# train['석식계'][973] = 478.8605851979346
# #train['석식계'][993] = 398

In [59]:
# 금요일이 랭크5라서 석식계가 train의 평균보다 높게 나오는 경향이 있어 train의 금요일을 10씩 낮추어주었다.
# train.loc[(train.요일==4), '석식계'] -= 10

### feature 선택

In [60]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [61]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '식사가능자수', '월(중식)',
       '월(석식)', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2'],
      dtype='object')

In [62]:
train.head(1)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,식사가능자수,월(중식),월(석식),공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2016-02-01,0,2601,50,150,238,0.0,1039.0,331.0,1,2016,2,1,2551.0,4,2,1,0,0


In [63]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사정원수', '본사휴가자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','월(중식)', '일','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사정원수', '본사휴가자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','월(석식)', '일','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사정원수', '본사휴가자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','월(중식)', '일','본사출장자수']] #중식계
x2_test = test[['요일(석식)', '본사정원수', '본사휴가자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','월(석식)', '일','본사출장자수']] #석식계

In [64]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.9142269	total: 11.6ms	remaining: 11.6s
1:	learn: 198.7669626	total: 13.9ms	remaining: 6.95s
2:	learn: 194.0739598	total: 16.1ms	remaining: 5.35s
3:	learn: 189.2655461	total: 18.4ms	remaining: 4.58s
4:	learn: 184.5524074	total: 20.8ms	remaining: 4.14s
5:	learn: 179.9470635	total: 23ms	remaining: 3.81s
6:	learn: 176.1620568	total: 25.4ms	remaining: 3.6s
7:	learn: 172.1794415	total: 27.6ms	remaining: 3.42s
8:	learn: 168.6142666	total: 29.9ms	remaining: 3.29s
9:	learn: 165.0547660	total: 32.1ms	remaining: 3.18s
10:	learn: 161.5470145	total: 34.4ms	remaining: 3.1s
11:	learn: 158.1776423	total: 36.7ms	remaining: 3.02s
12:	learn: 155.0692924	total: 38.9ms	remaining: 2.95s
13:	learn: 151.9935037	total: 41.4ms	remaining: 2.92s
14:	learn: 149.1817747	total: 43.9ms	remaining: 2.88s
15:	learn: 146.6950582	total: 46.2ms	remaining: 2.84s
16:	learn: 143.9755222	total: 48.5ms	remaining: 2.81s
17:	learn: 141.3970908	total: 50.9ms	remaining: 2.77s
18:	learn: 1

In [65]:
submission

,일자,중식계,석식계
0,2021-01-27,1019.481554,203.964873
1,2021-01-28,925.414856,430.920101
2,2021-01-29,580.861936,235.487363
3,2021-02-01,1203.452402,556.381578
4,2021-02-02,931.818735,428.070654
5,2021-02-03,912.993359,412.704140
6,2021-02-04,839.814410,373.933756
7,2021-02-05,616.805334,355.337598
8,2021-02-08,1190.878657,572.284758
9,2021-02-09,977.903073,519.474522


### export

In [66]:
submission.to_csv('qbran.csv', index=False)

In [67]:
train.iloc[1,2] = 498